In [24]:
import pandas as pd
import numpy as np


In [25]:
pno='1'

In [26]:
df=pd.read_csv('data/parking'+pno+'.csv')
df.head()

,Capacity,Occupancy,LastUpdated
0,1200,237,2021-10-04 07:59:42
1,1200,315,2021-10-04 08:25:42
2,1200,441,2021-10-04 08:59:42
3,1200,552,2021-10-04 09:32:46
4,1200,638,2021-10-04 09:59:48


In [27]:
df['LastUpdated']=pd.to_datetime(df['LastUpdated'])
df['PercentOccupied']=df['Occupancy']/df['Capacity']
df['date'] = df.LastUpdated.dt.date
df['dayofweek'] = df.LastUpdated.dt.dayofweek
df['hour'] = df['LastUpdated'].dt.hour
df['minute'] = df['LastUpdated'].dt.minute
df['hour_min'] = round(df['hour'] + (df['minute'] / 60), 1)

In [28]:
df.Occupancy=df.apply(lambda x:max(0,min(x['Capacity'],x['Occupancy'])),axis=1)
df.PercentOccupied=df.Occupancy/df.Capacity
df.dropna()
print('Min percernt {:.2%}'.format(df.PercentOccupied.min()))
print('Max percernt {:.2%}'.format(df.PercentOccupied.max()))

Min percernt 4.58%
Max percernt 79.50%


In [29]:
df['a1'] = np.where(df['PercentOccupied']<0.5,'p1','')
df['a2'] = np.where((df['PercentOccupied']>=0.5)&(df['PercentOccupied']<0.75),'p2','')
df['a3'] = np.where((df['PercentOccupied']>=0.75)&(df['PercentOccupied']<0.90),'p3','')
df['a4'] = np.where((df['PercentOccupied']>0.9),'p4','')
df['availability']=df['a1']+df['a2']+df['a3']+df['a4']
df=df.drop(['a1','a2','a3','a4'],axis=1)

In [30]:
df_train=df.copy()
df_train=df_train.drop(['LastUpdated','date','Capacity','Occupancy','PercentOccupied'],axis=1)
df_train

,dayofweek,hour,minute,hour_min,availability
0,0,7,59,8.0,p1
1,0,8,25,8.4,p1
2,0,8,59,9.0,p1
3,0,9,32,9.5,p1
4,0,9,59,10.0,p2
...,...,...,...,...,...
1307,6,14,30,14.5,p2
1308,6,15,3,15.0,p2
1309,6,15,29,15.5,p2
1310,6,16,3,16.0,p2


In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(
    
    steps=[("scaler", StandardScaler())]
)



# Create a `ColumnTransfomer` object 
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, ["hour_min","dayofweek","hour",'minute'])
        
    ]
)

# Append a Logistic Regression classifier
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier",LogisticRegression(multi_class='multinomial', solver='lbfgs'))]
)

In [32]:
from sklearn.model_selection import  train_test_split
from sklearn.metrics import classification_report


X=df_train.drop(columns='availability')
y=df_train['availability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          p1       0.77      0.72      0.74       120
          p2       0.78      0.83      0.80       143

    accuracy                           0.78       263
   macro avg       0.78      0.77      0.77       263
weighted avg       0.78      0.78      0.77       263



In [33]:
import pickle
filename='model/parking'+pno+'.sav'
pickle.dump(clf, open(filename, 'wb'))

In [34]:
date, time = '13/7/2022', '21:40'
predict_df = pd.DataFrame({'date': date + ' ' + time}, index=[0])
predict_df['date'] = pd.to_datetime(predict_df['date'])
df=predict_df
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['hour_min'] = round(df['hour'] + (df['minute'] / 60), 1)
df['dayofweek']= df['date'].dt.dayofweek
df.drop(['hour','minute','date'],axis=1)
predict_df=df
print(f"Priority For this lot: {clf.predict(predict_df)[0]}")



Priority For this lot: p2
